In [38]:
from dotenv import load_dotenv
import time
load_dotenv()
import os
from langchain_openai import ChatOpenAI
from langchain_community.chat_models import ChatOllama
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import OpenAIEmbeddings
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.messages import HumanMessage
from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from module import extract_news, summarize_news

In [40]:
openai_key = os.environ["OPENAI_API_KEY"]
vigilance_key = os.environ["VIGILENCE_API_KEY"]

In [41]:
gpt_turbo = ChatOpenAI(model_name="gpt-3.5-turbo")

In [58]:
news = extract_news(vigilance_key)
summary = summarize_news(gpt_turbo, news)

In [59]:
summary

AIMessage(content='Alerte météorologique pour la région de la pointe bretonne en raison de la tempête nommée "PIERRICK". Des vents forts et des vagues importantes sont attendus, avec un risque de submersion sur les littoraux. Des vigilances orange et jaune sont en place pour différents départements côtiers. La tempête se déplacera vers l\'est, affectant d\'autres régions avec des rafales de vent intenses. Il est conseillé de rester vigilant en raison de la conjonction de grandes marées et de conditions météorologiques extrêmes.', response_metadata={'token_usage': {'completion_tokens': 134, 'prompt_tokens': 1357, 'total_tokens': 1491}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_b28b39ffa8', 'finish_reason': 'stop', 'logprobs': None})

In [48]:
path = "data/Vigilance Table.csv"
loader = CSVLoader(file_path=path)
data = loader.load()


In [49]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(data)

In [50]:
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

In [51]:
retriever = vectorstore.as_retriever()

In [52]:
with open("contextualize_system_prompt.txt", "r") as files:
    contextualize_system_prompt = files.read()

with open("instruct.txt", "r") as files:
    instruct = files.read()

In [53]:
contextualize_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_system_prompt),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{question}"),
    ]
)
contextualize_prompt.pretty_print()
contextualize_chain = contextualize_prompt | gpt_turbo | StrOutputParser()

================================ System Message ================================

Given a chat history and the latest user question which might reference context in the chat history, formulate a standalone question just reformulate it if needed and otherwise return it as is.


============================= Messages Placeholder =============================

{chat_history}

================================ Human Message =================================

{question}


In [54]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", instruct),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{question}"),
    ]
)
prompt.pretty_print()

def contextualized_question(input: dict):
    if input.get("chat_history"):
        return contextualize_chain
    else:
        return input["question"]


rag_chain = (
    RunnablePassthrough.assign(
        context=(contextualized_question | retriever)
    )
    | prompt
    | gpt_turbo
)

================================ System Message ================================

Tu es un chatbot spécialiser sur les conseils liés aux intempéries qui ne répond UNIQUEMENT qu'en FRANÇAIS,
Voici les informations de vigilance d'aujourd'hui et de demain : {vigilance}
Tu peux t'aider du context suivant pour répondre : {context}


============================= Messages Placeholder =============================

{chat_history}

================================ Human Message =================================

{question}


In [60]:
chat_history = []
question = ("Quels sont les risques dans le finistère")
ai_msg = rag_chain.invoke({"question": question, "chat_history": chat_history, 'vigilance':summary})
print(ai_msg.content)

Dans le Finistère, les risques liés à la tempête nommée "PIERRICK" sont principalement des vents forts et des vagues importantes, avec un risque de submersion sur les littoraux. Des dégâts importants sont localement à craindre sur l'habitat léger et les installations provisoires sur le rivage et le littoral. Des submersions importantes sont à craindre aux abords des estuaires en période de marée haute de fort coefficient (vives eaux). Il est donc conseillé de se tenir éloigné des côtes et des estuaires, de surveiller la montée des eaux, de protéger les biens qui peuvent être inondés, et de rejoindre le plus haut point possible ou de se réfugier en étage, voire sur le toit en dernier recours.


In [61]:
chat_history = [HumanMessage(content=question), ai_msg]
question = "Que dois-je faire pour me protéger"
ai_msg = rag_chain.invoke({"question": question, "chat_history": chat_history, 'vigilance':summary})
print(ai_msg.content)

Pour vous protéger lors de la tempête "PIERRICK" dans le Finistère, voici quelques recommandations à suivre :

1. Protégez votre maison et les biens exposés au vent en renforçant les ouvertures et en sécurisant les objets extérieurs qui pourraient être emportés par le vent.
   
2. Renseignez-vous régulièrement auprès des autorités locales sur l'évolution de la situation météorologique et les consignes de sécurité à suivre.

3. Limitez vos déplacements au strict nécessaire pour éviter de vous mettre en danger inutilement.

4. Soyez prudent face aux chutes d'arbres et d'objets qui pourraient être projetés par le vent. Évitez de vous promener en forêt ou sous des arbres fragilisés.

5. Ne montez pas sur les toits pour effectuer des réparations ou des vérifications pendant la tempête. Attendez que les conditions météorologiques se stabilisent.

6. Si vous utilisez un groupe électrogène, veillez à l'installer à l'extérieur de la maison pour éviter les risques d'intoxication au monoxyde de c

In [62]:
chat_history = [HumanMessage(content=question), ai_msg]
question = "Comment me protéger de pierreck en lorainne"
ai_msg = rag_chain.invoke({"question": question, "chat_history": chat_history, 'vigilance':summary})
print(ai_msg.content)

Je suis désolé, mais je ne dispose pas d'informations spécifiques sur les mesures de protection à prendre en Lorraine pour faire face à la tempête "PIERRICK" car les informations que j'ai reçues concernent principalement la région de la pointe bretonne. Cependant, je peux vous fournir des conseils généraux pour vous protéger lors d'une tempête :

1. Restez informé de l'évolution de la situation météorologique en écoutant les bulletins d'information et en suivant les consignes des autorités locales.

2. Protégez votre maison en renforçant les ouvertures, en sécurisant les objets extérieurs et en évitant de vous rendre sur les toits pendant la tempête.

3. Limitez vos déplacements au strict nécessaire pour éviter de vous mettre en danger sur la route.

4. Éloignez-vous des arbres et des structures fragiles qui pourraient être endommagées par le vent.

5. Évitez les zones côtières et les endroits exposés aux vagues et aux risques de submersion.

En suivant ces conseils généraux de sécurit